In [50]:
## make imports from pa_lib possible (parent directory of file's directory)
import sys
from pathlib import Path

file_dir = Path.cwd()
parent_dir = file_dir.parent
sys.path.append(str(parent_dir))

%load_ext autoreload
%autoreload

import pandas as pd
import numpy as np
from dataclasses import dataclass


from pa_lib.file import (
    project_dir,
    load_bin,
    load_pickle,
    load_xlsx,
    store_bin,
)
from pa_lib.data import (
    as_dtype,
    dtFactor,
    lookup,
    desc_col,
    chi2_expected,
    clean_up_categoricals,
)
from pa_lib.util import cap_words, collect, value, normalize_rows, as_percent, flat_list
from pa_lib.html import HR, html, H1, H2, H3

# display long columns completely, show more rows
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load data sets

In [2]:
with project_dir("axinova"):
    ax_data = load_bin("ax_data.feather")
    spr_data = load_pickle("spr_data.pkl")
    time_codes = load_pickle("time_code_ratios.pkl")
    station_codes = load_pickle("station_code_ratios.pkl")
    global_codes = load_pickle("global_code_ratios.pkl")
    population_codes = load_pickle("population_ratios.pkl")

13:50:36 [INFO] Started loading binary file ...
13:50:36 [INFO] Reading from file C:\Users\kpf\data\axinova\ax_data.feather
13:50:36 [INFO] ... finished loading binary file in 0.35s (1.27s CPU)
13:50:36 [INFO] Started loading pickle file ...
13:50:36 [INFO] Reading from file C:\Users\kpf\data\axinova\spr_data.pkl
13:50:36 [INFO] ... finished loading pickle file in 0.01s (0.0s CPU)
13:50:36 [INFO] Started loading pickle file ...
13:50:36 [INFO] Reading from file C:\Users\kpf\data\axinova\time_code_ratios.pkl
13:50:36 [INFO] ... finished loading pickle file in 0.01s (0.02s CPU)
13:50:36 [INFO] Started loading pickle file ...
13:50:36 [INFO] Reading from file C:\Users\kpf\data\axinova\station_code_ratios.pkl
13:50:36 [INFO] ... finished loading pickle file in 0.0s (0.0s CPU)
13:50:36 [INFO] Started loading pickle file ...
13:50:36 [INFO] Reading from file C:\Users\kpf\data\axinova\global_code_ratios.pkl
13:50:36 [INFO] ... finished loading pickle file in 0.0s (0.0s CPU)
13:50:36 [INFO] St

## Select Axinova data by any column(s)

In [3]:
def _check_selection(data, selection, allowed_columns):
    allowed_values = {}
    for column in allowed_columns:
        allowed_values[column] = data[column].cat.categories
    if set(selection.keys()) - set(allowed_columns) != set():
        raise NameError(f"Illegal column name in selection: {selection.keys()}")
    clean_selection = {}
    for column in allowed_columns:
        if column in selection:
            col_values = flat_list(selection[column])
            if set(col_values) - set(allowed_values[column]) != set():
                raise ValueError(
                    f"Illegal value(s) in parameter {column}: {col_values}"
                )
            clean_selection[column] = col_values
        else:
            clean_selection[column] = None
    return clean_selection


def select_data(all_data, **selection):
    select_columns = (
        "DayOfWeek Station Variable Month TimeSlot Hour "
        + "Time TimeSlot_cat StationSprache Code".split()
    )
    selection = _check_selection(all_data, selection, allowed_columns=select_columns)
    row_mask = pd.Series([True] * all_data.shape[0])
    for col in select_columns:
        if selection[col] is not None:
            row_mask &= all_data[col].isin(selection[col])
    return all_data.loc[row_mask].pipe(clean_up_categoricals).reset_index(drop=True)

# Look up code ratios from Axinova data

In [4]:
RatioTable = pd.DataFrame


@dataclass
class Ratios:
    actual: RatioTable
    expected: RatioTable
    sd: RatioTable = None

In [5]:
def ax_population_ratios(variable: str, percent: bool = False) -> RatioTable:
    ratios = population_codes.query("Variable == @variable").pivot_table(
        values="Pop_Ratio", index="Variable", columns="Code"
    )
    return as_percent(ratios) if percent else ratios


def ax_global_ratios(variable: str, percent: bool = False) -> RatioTable:
    ratios = global_codes.query("Variable == @variable").pivot_table(
        values="Ratio", index="Variable", columns="Code"
    )
    return as_percent(ratios) if percent else ratios


def ax_station_ratios(variable: str, percent: bool = False) -> Ratios:
    actual_ratios = station_codes.query("Variable == @variable").pivot_table(
        values="Ratio", index="Station", columns="Code", fill_value=0
    )
    expected_ratios = ax_global_ratios(variable)
    if percent:
        result = Ratios(
            actual=as_percent(actual_ratios), expected=as_percent(expected_ratios)
        )
    else:
        result = Ratios(actual=actual_ratios, expected=expected_ratios)
    return result

In [6]:
def ax_ratios(
    variable: str,
    stations: str,
    reference: str = "all_stations",
    time_scale: str = "Hour",
    percent: bool = False,
) -> Ratios:
    subset = ax_data.loc[
        ax_data.Station.isin(flat_list(stations)) & (ax_data.Variable == variable)
    ]
    full_index = [
        (weekday, time)
        for weekday in ax_data["DayOfWeek"].cat.categories
        for time in ax_data[time_scale].cat.categories
    ]
    actual_counts = subset.pivot_table(
        values="Value",
        index=["DayOfWeek", time_scale],
        columns="Code",
        fill_value=0,
        aggfunc="sum",
    )
    actual_counts_sd_ratios = (
        (np.sqrt(actual_counts) / actual_counts)
        .fillna(0)
        .reindex(full_index, fill_value=0)
    )
    actual_ratios = normalize_rows(actual_counts).reindex(full_index, fill_value=0)

    if reference == "all_stations":
        expected_ratios = (
            time_codes[time_scale]
            .query("Variable == @variable")
            .pivot_table(
                values="Ratio",
                index=["DayOfWeek", time_scale],
                columns="Code",
                fill_value=0,
            )
        )
    elif reference == "station":
        expected_counts = chi2_expected(actual_counts)
        expected_ratios = normalize_rows(expected_counts).reindex(
            full_index, fill_value=0
        )
    else:
        raise ValueError(
            "Parameter 'reference' must be one of "
            + f"('station', 'all_stations'), was '{reference}'"
        )

    if percent:
        result = Ratios(
            actual=as_percent(actual_ratios),
            expected=as_percent(expected_ratios),
            sd=as_percent(actual_counts_sd_ratios),
        )
    else:
        result = Ratios(
            actual=actual_ratios, expected=expected_ratios, sd=actual_counts_sd_ratios
        )
    return result

# Look up SPR+ data split by variable

In [7]:
def spr_split(
    stations,
    variable,
    reference="station",
    time_scale="Hour",
    type="abs",
    incl_totals=True,
    decimals=1,
):
    if type not in ["abs", "rel"]:
        raise ValueError(
            f"Parameter 'type' must be one of ('abs', 'rel'), was '{type}'"
        )
    if reference not in ["station", "all_stations"]:
        raise ValueError(
            "Parameter 'reference' must be one of "
            + f"('station', 'all_stations'), was '{reference}'"
        )

    spr_counts = (
        spr_data.loc[spr_data.Station.isin(flat_list(stations))]
        .groupby(["DayOfWeek", time_scale])[["Total"]]
        .agg("sum")
    )
    spr_count_sd_ratios = (np.sqrt(spr_counts) / spr_counts).fillna(0)

    ratios = ax_ratios(
        stations=stations,
        variable=variable,
        reference=reference,
        time_scale=time_scale,
        percent=False,
    )
    if type == "abs":
        code_ratios = ratios.actual
    elif type == "rel":
        code_ratios = ratios.actual - ratios.expected
    split_counts = code_ratios.mul(spr_counts.Total, axis="index").round(decimals)

    sd_ratios = np.sqrt(
        (ratios.sd ** 2).add((spr_count_sd_ratios ** 2).values, axis="index")
    )

    if incl_totals:
        split_counts.set_axis(split_counts.columns.to_list(), axis="columns", inplace=True)
        split_counts["Total"] = spr_counts.round(decimals)
        sd_ratios.set_axis(sd_ratios.columns.to_list(), axis="columns", inplace=True)
        sd_ratios["Total"] = spr_count_sd_ratios.values
    if decimals == 0:
        split_counts = split_counts.astype("int")

    return split_counts, sd_ratios

# Tests

## Station ratios

In [8]:
with value(ax_station_ratios(variable="g_220", percent=True)) as autobesitz:
    display(autobesitz.actual)
    display(autobesitz.expected)
    display(autobesitz.actual.sub(autobesitz.expected.values, axis="columns"))

Code,1 Auto,2+ Autos,Keines
Station,,,
Aarau,43,30,27
Basel SBB,44,20,36
Bellinzona,52,38,11
Bern,44,28,28
Biel/Bienne,50,25,25
Brig,52,39,9
Chur,46,37,17
Fribourg,31,56,13
Genève Aéroport,42,49,10


Code,1 Auto,2+ Autos,Keines
Variable,,,
g_220,45,29,26


Code,1 Auto,2+ Autos,Keines
Station,,,
Aarau,-2,1,1
Basel SBB,-1,-9,10
Bellinzona,7,9,-15
Bern,-1,-1,2
Biel/Bienne,5,-4,-1
Brig,7,10,-17
Chur,1,8,-9
Fribourg,-14,27,-13
Genève Aéroport,-3,20,-16


## Time ratios for one station

In [9]:
with value(ax_ratios(stations="Lausanne", variable="g_220", time_scale="TimeSlot")) as lausanne_auto:
    display(lausanne_auto.actual)
    display(lausanne_auto.expected)
    display(lausanne_auto.actual - lausanne_auto.expected)

Code                     1 Auto  2+ Autos    Keines
DayOfWeek TimeSlot                                 
Monday    Nacht        0.312328  0.477729  0.209943
          Morgen-Rush  0.484154  0.309249  0.206596
          Morgen       0.541461  0.327835  0.130704
          Mittag       0.576887  0.232601  0.190512
          Nachmittag   0.446346  0.397710  0.155944
          Abend-Rush   0.426679  0.407598  0.165723
          Abend        0.484624  0.238380  0.276997
Tuesday   Nacht        0.280834  0.594073  0.125092
          Morgen-Rush  0.551503  0.243898  0.204599
          Morgen       0.421252  0.374605  0.204143
          Mittag       0.559244  0.301280  0.139477
          Nachmittag   0.478397  0.352437  0.169166
          Abend-Rush   0.440691  0.350291  0.209018
          Abend        0.346230  0.312548  0.341221
Wednesday Nacht        0.294283  0.414411  0.291306
          Morgen-Rush  0.506339  0.325127  0.168534
          Morgen       0.396389  0.430390  0.173220
          Mittag       0.490246  0.380846  0.128908
          Nachmittag   0.428580  0.420290  0.151130
          Abend-Rush   0.429823  0.354348  0.215829
          Abend        0.313845  0.350778  0.335377
Thursday  Nacht        0.372253  0.520915  0.106832
          Morgen-Rush  0.528235  0.299180  0.172585
          Morgen       0.486035  0.410599  0.103366
          Mittag       0.554989  0.282066  0.162944
          Nachmittag   0.437111  0.397374  0.165514
          Abend-Rush   0.414507  0.380994  0.204498
          Abend        0.327595  0.433923  0.238483
Friday    Nacht        0.247650  0.452111  0.300238
          Morgen-Rush  0.496061  0.284759  0.219180
          Morgen       0.562418  0.350884  0.086698
          Mittag       0.467447  0.415013  0.117540
          Nachmittag   0.443630  0.364158  0.192212
          Abend-Rush   0.413449  0.375093  0.211457
          Abend        0.442978  0.280487  0.276535
Saturday  Nacht        0.387935  0.405683  0.206382
          Morgen-Rush  0.603718  0.231880  0.164402
          Morgen       0.431590  0.331242  0.237168
          Mittag       0.382958  0.265585  0.351456
          Nachmittag   0.297112  0.421475  0.281413
          Abend-Rush   0.417537  0.301823  0.280640
          Abend        0.341891  0.329451  0.328658
Sunday    Nacht        0.306510  0.248231  0.445259
          Morgen-Rush  0.496771  0.343634  0.159595
          Morgen       0.320690  0.301962  0.377348
          Mittag       0.407254  0.296425  0.296321
          Nachmittag   0.446408  0.297996  0.255596
          Abend-Rush   0.360335  0.289068  0.350597
          Abend        0.341091  0.444738  0.214170

Code                     1 Auto  2+ Autos    Keines
DayOfWeek TimeSlot                                 
Monday    Nacht        0.430578  0.286991  0.282432
          Morgen-Rush  0.477160  0.289778  0.233061
          Morgen       0.427002  0.296161  0.276837
          Mittag       0.426865  0.304519  0.268616
          Nachmittag   0.443431  0.297881  0.258688
          Abend-Rush   0.484882  0.281929  0.233189
          Abend        0.438607  0.250351  0.311042
Tuesday   Nacht        0.423316  0.299197  0.277487
          Morgen-Rush  0.493769  0.285770  0.220461
          Morgen       0.409663  0.330026  0.260311
          Mittag       0.453954  0.303390  0.242656
          Nachmittag   0.445387  0.309361  0.245252
          Abend-Rush   0.474017  0.284430  0.241553
          Abend        0.432994  0.276751  0.290255
Wednesday Nacht        0.375680  0.304505  0.319816
          Morgen-Rush  0.486070  0.285918  0.228012
          Morgen       0.405284  0.336410  0.258306
          Mittag       0.432810  0.327270  0.239920
          Nachmittag   0.432022  0.327774  0.240204
          Abend-Rush   0.469448  0.280658  0.249894
          Abend        0.425482  0.291959  0.282559
Thursday  Nacht        0.422813  0.299727  0.277460
          Morgen-Rush  0.502530  0.275531  0.221938
          Morgen       0.443002  0.329958  0.227041
          Mittag       0.439880  0.317049  0.243071
          Nachmittag   0.466144  0.301178  0.232678
          Abend-Rush   0.476400  0.279863  0.243738
          Abend        0.451126  0.282667  0.266207
Friday    Nacht        0.409063  0.324866  0.266070
          Morgen-Rush  0.481159  0.288792  0.230050
          Morgen       0.455911  0.292137  0.251952
          Mittag       0.429915  0.325216  0.244869
          Nachmittag   0.436277  0.303831  0.259892
          Abend-Rush   0.445822  0.296858  0.257320
          Abend        0.428789  0.296652  0.274559
Saturday  Nacht        0.374828  0.363267  0.261905
          Morgen-Rush  0.464483  0.280987  0.254529
          Morgen       0.446119  0.252826  0.301055
          Mittag       0.443046  0.258726  0.298228
          Nachmittag   0.430801  0.287824  0.281374
          Abend-Rush   0.439294  0.257653  0.303053
          Abend        0.439709  0.281985  0.278306
Sunday    Nacht        0.354114  0.362091  0.283794
          Morgen-Rush  0.436314  0.275097  0.288589
          Morgen       0.437888  0.238541  0.323571
          Mittag       0.435676  0.246578  0.317746
          Nachmittag   0.433843  0.248868  0.317289
          Abend-Rush   0.410767  0.273526  0.315707
          Abend        0.386540  0.313053  0.300407

Code                     1 Auto  2+ Autos    Keines
DayOfWeek TimeSlot                                 
Monday    Nacht       -0.118249  0.190738 -0.072488
          Morgen-Rush  0.006994  0.019471 -0.026465
          Morgen       0.114459  0.031674 -0.146133
          Mittag       0.150022 -0.071918 -0.078104
          Nachmittag   0.002915  0.099829 -0.102744
          Abend-Rush  -0.058203  0.125669 -0.067466
          Abend        0.046017 -0.011971 -0.034046
Tuesday   Nacht       -0.142481  0.294876 -0.152395
          Morgen-Rush  0.057733 -0.041871 -0.015862
          Morgen       0.011589  0.044579 -0.056169
          Mittag       0.105290 -0.002110 -0.103179
          Nachmittag   0.033011  0.043075 -0.076086
          Abend-Rush  -0.033326  0.065861 -0.032535
          Abend       -0.086764  0.035798  0.050966
Wednesday Nacht       -0.081396  0.109906 -0.028510
          Morgen-Rush  0.020269  0.039209 -0.059478
          Morgen      -0.008895  0.093980 -0.085086
          Mittag       0.057436  0.053576 -0.111012
          Nachmittag  -0.003442  0.092516 -0.089074
          Abend-Rush  -0.039626  0.073690 -0.034065
          Abend       -0.111637  0.058819  0.052818
Thursday  Nacht       -0.050560  0.221188 -0.170628
          Morgen-Rush  0.025704  0.023649 -0.049353
          Morgen       0.043033  0.080642 -0.123675
          Mittag       0.115109 -0.034982 -0.080127
          Nachmittag  -0.029033  0.096197 -0.067164
          Abend-Rush  -0.061892  0.101132 -0.039239
          Abend       -0.123531  0.151255 -0.027724
Friday    Nacht       -0.161413  0.127245  0.034168
          Morgen-Rush  0.014903 -0.004033 -0.010870
          Morgen       0.106507  0.058747 -0.165254
          Mittag       0.037532  0.089797 -0.127329
          Nachmittag   0.007353  0.060327 -0.067680
          Abend-Rush  -0.032373  0.078235 -0.045862
          Abend        0.014189 -0.016164  0.001975
Saturday  Nacht        0.013107  0.042416 -0.055523
          Morgen-Rush  0.139235 -0.049107 -0.090127
          Morgen      -0.014530  0.078416 -0.063886
          Mittag      -0.060088  0.006859  0.053229
          Nachmittag  -0.133690  0.133651  0.000039
          Abend-Rush  -0.021757  0.044169 -0.022413
          Abend       -0.097818  0.047465  0.050352
Sunday    Nacht       -0.047604 -0.113860  0.161464
          Morgen-Rush  0.060456  0.068538 -0.128994
          Morgen      -0.117198  0.063422  0.053777
          Mittag      -0.028421  0.049847 -0.021426
          Nachmittag   0.012565  0.049128 -0.061693
          Abend-Rush  -0.050432  0.015542  0.034890
          Abend       -0.045449  0.131686 -0.086237

## SPR+ data for one station split by ratios

In [10]:
with value(("Bern", "g_220", "TimeSlot")) as (stat, var, scale):
    display(spr_split(stations=stat, variable=var, type="abs", time_scale=scale)[0])
    display(spr_split(stations=stat, variable=var, type="rel", time_scale=scale)[0])

1 Auto  2+ Autos   Keines     Total
DayOfWeek TimeSlot                                         
Monday    Nacht         8505.2    1544.8   3018.0   13068.0
          Morgen-Rush  44651.2   33956.5  29179.2  107786.9
          Morgen       22499.7   14247.7  16554.1   53301.5
          Mittag       26327.0   15745.0  23984.1   66056.2
          Nachmittag   40273.5   24309.4  32390.9   96973.8
          Abend-Rush   73325.8   51155.7  43166.3  167647.9
          Abend        33548.9   19116.1  34881.2   87546.3
Tuesday   Nacht         7212.6    3355.0   2376.0   12943.6
          Morgen-Rush  48020.4   30194.5  32871.2  111086.1
          Morgen       18268.4   15366.1  17544.1   51178.6
          Mittag       28359.2   20912.4  14236.6   63508.2
          Nachmittag   38503.2   30925.8  24565.1   93994.1
          Abend-Rush   74547.7   58016.2  38521.3  171085.2
          Abend        40441.4   19689.0  30489.4   90619.8
Wednesday Nacht         8914.2    2711.9   2196.8   13823.0
          Morgen-Rush  44638.1   29349.8  32071.5  106059.4
          Morgen       14387.4   21954.5  13938.3   50280.1
          Mittag       24705.3   20548.9  19517.5   64771.7
          Nachmittag   38521.6   31879.8  26412.7   96814.2
          Abend-Rush   74757.9   50611.3  42724.2  168093.5
          Abend        46324.3   16629.4  29907.7   92861.5
Thursday  Nacht         9190.4    2705.4   3518.9   15414.6
          Morgen-Rush  47147.0   23014.6  34547.9  104709.5
          Morgen       18270.6   15461.9  16947.3   50679.8
          Mittag       22629.7   20354.3  20588.9   63572.8
          Nachmittag   39555.4   30804.6  25030.0   95390.1
          Abend-Rush   74057.4   46602.5  47065.4  167725.2
          Abend        44616.2   21262.3  30827.5   96706.1
Friday    Nacht        10666.2    5736.4   5361.0   21763.6
          Morgen-Rush  42639.1   24330.0  22746.7   89715.8
          Morgen       22809.8   12872.1  12691.4   48373.3
          Mittag       30501.7   16126.0  15854.6   62482.3
          Nachmittag   45431.0   25583.1  29520.3  100534.3
          Abend-Rush   71832.2   54020.7  36960.9  162813.8
          Abend        42333.2   25367.6  38349.7  106050.5
Saturday  Nacht        13666.8   12471.7   6052.9   32191.4
          Morgen-Rush  16384.5   14824.8  13748.4   44957.7
          Morgen       22526.2   18315.2  14743.6   55585.0
          Mittag       33488.8   20142.5  18051.0   71682.3
          Nachmittag   66989.4   34257.8  24839.0  126086.2
          Abend-Rush   63183.9   40299.7  36905.4  140389.0
          Abend        54686.6   29165.7  28635.3  112487.6
Sunday    Nacht         9927.8   12147.3   4608.8   26683.9
          Morgen-Rush  18652.6    5738.5   6263.3   30654.4
          Morgen       20904.9   11808.8  17640.1   50353.8
          Mittag       31830.2   14898.6  25272.8   72001.6
          Nachmittag   68699.0   24467.5  41988.8  135155.2
          Abend-Rush   72286.8   38200.7  43367.3  153854.8
          Abend        53194.4   25171.4  28605.5  106971.3

1 Auto  2+ Autos   Keines     Total
DayOfWeek TimeSlot                                         
Monday    Nacht         2794.5   -2144.7   -649.9   13068.0
          Morgen-Rush  -2451.1    3524.9  -1073.8  107786.9
          Morgen        -792.8    -801.0   1593.8   53301.5
          Mittag       -2539.1   -2904.8   5443.9   66056.2
          Nachmittag   -2103.5   -3069.4   5172.9   96973.8
          Abend-Rush      64.6    3823.4  -3888.0  167647.9
          Abend        -4708.3   -5601.0  10309.3   87546.3
Tuesday   Nacht         1556.3    -299.4  -1256.9   12943.6
          Morgen-Rush   -523.6   -1168.6   1692.2  111086.1
          Morgen       -4096.4     916.7   3179.6   51178.6
          Mittag         606.5    2982.0  -3588.5   63508.2
          Nachmittag   -2571.7    4388.3  -1816.6   93994.1
          Abend-Rush    -215.6    9713.4  -9497.8  171085.2
          Abend          841.0   -5895.9   5054.8   90619.8
Wednesday Nacht         2873.7   -1190.8  -1682.9   13823.0
          Morgen-Rush  -1709.3    -594.1   2303.4  106059.4
          Morgen       -7584.8    7758.8   -174.0   50280.1
          Mittag       -3599.5    2261.7   1337.8   64771.7
          Nachmittag   -3785.6    4546.1   -760.5   96814.2
          Abend-Rush    1302.0    3153.2  -4455.2  168093.5
          Abend         5744.4   -9588.3   3843.9   92861.5
Thursday  Nacht         2454.3   -1646.7   -807.6   15414.6
          Morgen-Rush   1389.5   -6548.2   5158.7  104709.5
          Morgen       -3876.2    1153.4   2722.8   50679.8
          Mittag       -5151.3    2405.7   2745.6   63572.8
          Nachmittag   -2129.5    3872.9  -1743.5   95390.1
          Abend-Rush     762.4    -751.7    -10.7  167725.2
          Abend         2356.2   -6040.9   3684.7   96706.1
Friday    Nacht         1155.6    -408.2   -747.5   21763.6
          Morgen-Rush   3433.8    -999.6  -2434.2   89715.8
          Morgen        1671.0    -785.2   -885.7   48373.3
          Mittag        3197.3   -1514.8  -1682.5   62482.3
          Nachmittag    1498.1   -2801.0   1302.9  100534.3
          Abend-Rush     683.5    8053.2  -8736.7  162813.8
          Abend        -4010.2   -4573.8   8584.1  106050.5
Saturday  Nacht         -400.7    3383.0  -2982.4   32191.4
          Morgen-Rush  -3261.7    2131.8   1129.9   44957.7
          Morgen       -1764.2    2621.8   -857.6   55585.0
          Mittag        2164.0     -95.7  -2068.4   71682.3
          Nachmittag   11890.5   -1340.3 -10550.1  126086.2
          Abend-Rush    1834.7     663.4  -2498.1  140389.0
          Abend         5530.1   -2593.1  -2937.0  112487.6
Sunday    Nacht        -1732.9    4613.6  -2880.7   26683.9
          Morgen-Rush   5256.8   -2916.2  -2340.6   30654.4
          Morgen       -1099.4   -2407.7   3507.1   50353.8
          Mittag         365.9   -5429.7   5063.8   72001.6
          Nachmittag    9636.9  -13691.1   4054.2  135155.2
          Abend-Rush    5053.1   -5237.4    184.3  153854.8
          Abend         6448.6   -5030.0  -1418.6  106971.3

# Case study: Find best stations / time slots for owner of multiple cars

## Population ratios

In [11]:
ax_population_ratios(variable="g_220", percent=True)

Code,1 Auto,2+ Autos,Keines
Variable,,,
g_220,44,40,16


## Ratios over all stations

In [12]:
ax_global_ratios(variable="g_220", percent=True)

Code,1 Auto,2+ Autos,Keines
Variable,,,
g_220,45,29,26


Multi-car owners are **underrepresented** at digital station panels (29% vs. 40%)

## Ratios per station

In [13]:
ax_station_ratios(variable="g_220", percent=True).actual

Code,1 Auto,2+ Autos,Keines
Station,,,
Aarau,43,30,27
Basel SBB,44,20,36
Bellinzona,52,38,11
Bern,44,28,28
Biel/Bienne,50,25,25
Brig,52,39,9
Chur,46,37,17
Fribourg,31,56,13
Genève Aéroport,42,49,10


Best stations are **Fribourg** (56%), **Zürich Flughafen** (50%), and **Genève Aéroport** (49%).

This compares to 29% over all stations, and 40% in the base population.

## Time slots at one station: Zürich Flughafen

In [14]:
prozent = ax_ratios(
    variable="g_220", stations="Zürich Flughafen", time_scale="Hour", percent=True
).actual
personen = spr_split(
    stations="Zürich HB", variable="g_220", time_scale="Hour", type="abs", decimals=0
)[0]
time_analysis = prozent.merge(
    personen, left_index=True, right_index=True, suffixes=(" [%]", " [Pers.]")
)

time_analysis.loc[prozent["2+ Autos"] > 40]

1 Auto [%]  2+ Autos [%]  Keines [%]  1 Auto [Pers.]  \
DayOfWeek Hour                                                         
Monday    00            48            52           0            1123   
          01             0           100           0             802   
          04            18            82           0               0   
          05            29            60          11            1506   
          07            11            79          10           34559   
          09            38            59           3           14221   
          10            39            61           0           12515   
          11            44            52           4           17402   
          12            28            63           9           25315   
          13            31            67           3           19727   
          14            10            86           5           21865   
          15            34            58           7           25268   
          16            47            50           3           32116   
          19            18            49          33           26497   
          23            22            65          13            5848   
Tuesday   00             0           100           0            1211   
          04             0           100           0             491   
          05             6            79          15             955   
          06            18            62          19           13497   
          07            14            83           3           36538   
          08            33            63           3           31175   
          09            22            62          16           15898   
          10            40            48          12           13284   
          12            36            46          18           24729   
          13            16            72          12           21422   
          14            32            62           6           20304   
          15            38            56           6           18418   
          16            33            61           6           33664   
          18            25            46          30           42908   
          19            27            57          16           24160   
Wednesday 00            49            51           0             880   
          03             0           100           0             321   
          04            42            41          17             603   
          06            25            54          21           12555   
          07            15            65          21           33923   
          08            27            48          25           27911   
          10             7            87           6           15304   
          11            33            50          16           21522   
          12            51            46           3           25585   
          13            29            59          12           18061   
          14            43            52           5           22045   
          15            41            57           3           20312   
          16            41            49          11           29403   
          17            30            45          25           48907   
          18            19            68          14           36349   
          19            25            52          22           23459   
          20            39            46          15           18086   
          21            24            55          21           12219   
          22            42            55           3            7961   
          23            32            42          27            4302   
Thursday  00             0           100           0             982   
          03             0           100           0               0   
          04            31            69           0             597   
          07            40            56           5    

In [ ]:
time_analysis.sort_values("2+ Autos [Pers.]", ascending=False).head(20)

## Time slots with CI at one station: Zürich Flughafen

In [16]:
(counts, sd_ratios) = spr_split(
    stations="Zürich Flughafen", variable="g_220", time_scale="Hour", type="abs", decimals=1
)

counts.merge(
    as_percent(sd_ratios), left_index=True, right_index=True, suffixes=(" [Pers]", " [sd%]")
)

1 Auto [Pers]  2+ Autos [Pers]  Keines [Pers]  Total [Pers]  \
DayOfWeek Hour                                                                
Monday    00             11.3             12.3            0.0          23.6   
          01              0.0              0.0            0.0           0.0   
          02              0.0              0.0            0.0           0.0   
          03              0.0              0.0            0.0           0.0   
          04              0.0              0.0            0.0           0.0   
          05             17.1             35.5            6.8          59.5   
          06             85.7             72.7           75.0         233.4   
          07             56.8            414.8           55.1         526.6   
          08            289.5            137.2           67.0         493.8   
          09            124.5            191.7           10.3         326.4   
          10            114.7            179.9            0.0         294.5   
          11            145.6            174.1           14.4         334.1   
          12            122.4            274.6           39.2         436.3   
          13            114.6            247.7            9.9         372.2   
          14             34.9            306.8           16.7         358.3   
          15            138.1            234.5           28.7         401.2   
          16            265.9            279.7           15.2         560.8   
          17            349.1            302.9          114.1         766.1   
          18            197.5            187.5          245.3         630.2   
          19             73.6            202.8          133.8         410.2   
          20            135.8             76.9           69.8         282.5   
          21             68.8             52.4           74.3         195.5   
          22             14.2             37.2           81.7         133.1   
          23             14.8             44.9            8.8          68.5   
Tuesday   00              0.0             19.1            0.0          19.1   
          01              0.0              0.0            0.0           0.0   
          02              0.0              0.0            0.0           0.0   
          03              0.0              0.0            0.0           0.0   
          04              0.0              0.0            0.0           0.0   
          05              3.6             43.4            8.2          55.2   
          06             43.8            149.0           45.8         238.6   
          07             77.8            456.8           16.6         551.2   
          08            168.8            321.2           16.9         507.0   
          09             70.8            198.9           49.5         319.2   
          10            111.0            132.0           34.7         277.6   
          11            142.8            122.1           54.2         319.0   
          12            150.2            196.0           75.5         421.7   
          13             57.0            261.2           42.4         360.5   
          14            109.3            211.2           22.2         342.6   
          15            149.9            218.4           24.8         393.1   
          16            184.1            341.6           34.7         560.5   
          17            426.3            185.4          172.7         784.4   
          18            159.8            295.9          193.8         649.5   
          19            111.8            240.0           68.3         420.0   
          20            163.5             74.5           51.0         288.9   
          21             58.7             82.7           63.1         204.6   
          22             73.8             42.9           25.3         142.0   
          23             76.4              0.0            0.0          76.4   
Wednesday 00             10.8             11.1            0.0     

## Analysis for ZRH Airport (ax_data only)

In [51]:
with value(
    ax_ratios(
        variable="g_220",
        stations="Zürich Flughafen - Airside",
        reference="station",
        time_scale="TimeSlot",
    )
) as ratios:
    display(H3("Actual ratios"))
    display(ratios.actual)
    display(HR(), H3("Confidence intervals (sd in %)"))
    display(as_percent(ratios.sd))

Code                     1 Auto  2+ Autos    Keines
DayOfWeek TimeSlot                                 
Monday    Nacht        0.180987  0.552202  0.266811
          Morgen-Rush  0.273142  0.434854  0.292004
          Morgen       0.429608  0.413985  0.156407
          Mittag       0.160982  0.609423  0.229595
          Nachmittag   0.448211  0.480042  0.071747
          Abend-Rush   0.371419  0.478662  0.149919
          Abend        0.366501  0.421789  0.211709
Tuesday   Nacht        0.330247  0.669753  0.000000
          Morgen-Rush  0.505201  0.343728  0.151071
          Morgen       0.473495  0.470133  0.056372
          Mittag       0.281580  0.653515  0.064906
          Nachmittag   0.230271  0.685160  0.084569
          Abend-Rush   0.571696  0.278415  0.149889
          Abend        0.502403  0.389184  0.108413
Wednesday Nacht        0.318483  0.603296  0.078221
          Morgen-Rush  0.339760  0.506542  0.153698
          Morgen       0.441297  0.495463  0.063241
          Mittag       0.389913  0.542922  0.067165
          Nachmittag   0.287621  0.637168  0.075211
          Abend-Rush   0.380782  0.475836  0.143382
          Abend        0.426989  0.535527  0.037485
Thursday  Nacht        0.131547  0.638583  0.229871
          Morgen-Rush  0.474298  0.349632  0.176070
          Morgen       0.524723  0.362134  0.113143
          Mittag       0.498468  0.377932  0.123599
          Nachmittag   0.328184  0.509764  0.162052
          Abend-Rush   0.534097  0.373879  0.092024
          Abend        0.357598  0.477775  0.164627
Friday    Nacht        0.419110  0.536325  0.044564
          Morgen-Rush  0.382463  0.440346  0.177191
          Morgen       0.398114  0.587831  0.014055
          Mittag       0.422843  0.403243  0.173914
          Nachmittag   0.478173  0.428999  0.092828
          Abend-Rush   0.427678  0.308815  0.263506
          Abend        0.391722  0.364702  0.243576
Saturday  Nacht        0.345496  0.542165  0.112339
          Morgen-Rush  0.484003  0.381428  0.134569
          Morgen       0.355406  0.486557  0.158037
          Mittag       0.462163  0.426079  0.111758
          Nachmittag   0.456889  0.405917  0.137194
          Abend-Rush   0.464603  0.352769  0.182627
          Abend        0.531247  0.424358  0.044394
Sunday    Nacht        0.244906  0.621796  0.133298
          Morgen-Rush  0.348888  0.432377  0.218735
          Morgen       0.435467  0.329458  0.235076
          Mittag       0.504870  0.271625  0.223504
          Nachmittag   0.403811  0.412572  0.183618
          Abend-Rush   0.378084  0.404782  0.217134
          Abend        0.445524  0.384337  0.170139

Code                   1 Auto  2+ Autos  Keines
DayOfWeek TimeSlot                             
Monday    Nacht            27        16      22
          Morgen-Rush      15        12      15
          Morgen           17        18      29
          Mittag           26        13      22
          Nachmittag       13        12      32
          Abend-Rush       15        13      23
          Abend            14        13      18
Tuesday   Nacht            25        17       0
          Morgen-Rush      13        15      23
          Morgen           21        21      60
          Mittag           22        15      47
          Nachmittag       19        11      32
          Abend-Rush       15        22      30
          Abend            13        14      27
Wednesday Nacht            24        17      48
          Morgen-Rush      16        13      24
          Morgen           16        16      44
          Mittag           15        13      36
          Nachmittag       18        12      35
          Abend-Rush       15        13      25
          Abend            15        13      50
Thursday  Nacht            58        26      44
          Morgen-Rush      15        17      24
          Morgen           16        19      34
          Mittag           14        16      29
          Nachmittag       15        12      21
          Abend-Rush       12        14      29
          Abend            16        14      24
Friday    Nacht            16        14      50
          Morgen-Rush      12        11      18
          Morgen           19        15     100
          Mittag           17        17      26
          Nachmittag       12        13      27
          Abend-Rush       13        16      17
          Abend            13        14      17
Saturday  Nacht            24        19      42
          Morgen-Rush      11        13      21
          Morgen           15        13      22
          Mittag           12        13      25
          Nachmittag       12        12      21
          Abend-Rush       14        16      22
          Abend            13        15      47
Sunday    Nacht            28        17      37
          Morgen-Rush      16        14      20
          Morgen           15        17      20
          Mittag           10        14      15
          Nachmittag       10        10      15
          Abend-Rush       11        11      15
          Abend            11        12      17